In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [166]:
# Necessary methods
def divideDateRange(rangeStart, rangeEnd, interval):
    """
    Divides a date range to specific intervals. The process starts from the end date and
    goes backwards to reach the start date.

    Args:
        rangeStart, rangeEnd: datetime.datetime: Start and end of the desired date range.
        interval: datetime.timedelta: The desired interval to divide the date range to.
    
    Returns: 
        A list of dates.
    """
    lstOutput = []
    temp = rangeEnd
    while rangeStart < temp:
        lstOutput.append(temp)
        temp -= interval
    
    return lstOutput

def netGrowthRate(AuMLastDay, AuMFirstDay, annualized):
    """
    Returns the net growth rate of the fund's AuM 
    The formula: (Last day value - First day value)/First day value = growth rate

    Args:
        AuMLastDay, AuMFirstDay: float: AuM of the fund at the beginning and end of the interval
    
    Returns: 
        float.
    """
    if annualized:
        return np.power((AuMLastDay-AuMFirstDay)/AuMFirstDay + 1, 1/3) - 1
    else:
        return (AuMLastDay-AuMFirstDay)/AuMFirstDay

In [178]:
# Import the data for fund 1
f1_DailyAuM = pd.read_csv("./Data/Daily AuM - AuM (1).csv")
f1_DailyTrades = pd.read_csv("./Data/Daily AuM - Trades_10.csv")

# Get the union of both datasets (Date-wise)
f1_DailyAuM.Date = pd.to_datetime(f1_DailyAuM.Date, dayfirst = True, format = "mixed")
f1_DailyTrades.buy_date = pd.to_datetime(f1_DailyTrades.buy_date, dayfirst = True, format = "mixed")
f1_DailyTrades.sell_date = pd.to_datetime(f1_DailyTrades.sell_date, dayfirst = True, format = "mixed")

f1_DailyTrades_LastDate = f1_DailyTrades.iloc[-1].sell_date
f1_DailyAuM_LastDate = f1_DailyAuM.iloc[-1].Date

if f1_DailyTrades_LastDate < f1_DailyAuM_LastDate:
    f1_DailyAuM = f1_DailyAuM[f1_DailyAuM.Date <= f1_DailyTrades_LastDate]
else:
    f1_DailyTrades = f1_DailyTrades[f1_DailyTrades.sell_date <= f1_DailyAuM_LastDate]

# start and end of date of the entire dataset
START_DATE = f1_DailyAuM.iloc[0].Date
END_DATE   = f1_DailyAuM.iloc[-1].Date 

print(f"The entire date range is from {START_DATE} to {END_DATE} ({(f1_DailyAuM.iloc[-1].Date - f1_DailyAuM.iloc[0].Date)})")


The entire date range is from 2019-12-27 00:00:00 to 2023-10-31 00:00:00 (1404 days 00:00:00)


In [182]:
# Growth rate of fund net worth
# This metric is calculated on three intervals, six months, 1 year and 3 years
range_6Mo = divideDateRange(START_DATE, END_DATE, datetime.timedelta(days=30*6))
range_1Yr = divideDateRange(START_DATE, END_DATE, datetime.timedelta(days=30*12))
range_3Yrs = divideDateRange(START_DATE, END_DATE, datetime.timedelta(days=30*12*3))

tmpAuM_6Mo = f1_DailyAuM[(range_6Mo[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_6Mo[0])]
growthRate_6Mo = netGrowthRate(tmpAuM_6Mo.iloc[-1].AuM, tmpAuM_6Mo.iloc[0].AuM, False)

tmpAuM_1Yr = f1_DailyAuM[(range_1Yr[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_1Yr[0])]
growthRate_1Yr = netGrowthRate(tmpAuM_1Yr.iloc[-1].AuM, tmpAuM_1Yr.iloc[0].AuM, False)

tmpAuM_3Yrs = f1_DailyAuM[(range_3Yrs[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_3Yrs[0])]
growthRate_3Yrs = netGrowthRate(tmpAuM_3Yrs.iloc[-1].AuM, tmpAuM_3Yrs.iloc[0].AuM, False)

print(f"6 months growth rate (%) = {growthRate_6Mo*100} calculated from {tmpAuM_6Mo.iloc[-1].Date} to {tmpAuM_6Mo.iloc[0].Date}")
print(f"1 year growth rate (%) = {growthRate_1Yr*100} calculated from {tmpAuM_1Yr.iloc[-1].Date} to {tmpAuM_1Yr.iloc[0].Date}")
print(f"3 years growth rate (%) = {growthRate_3Yrs*100} calculated from {tmpAuM_3Yrs.iloc[-1].Date} to {tmpAuM_3Yrs.iloc[0].Date}")


6 months growth rate (%) = 4.3824640703169635 calculated from 2023-10-31 00:00:00 to 2023-05-05 00:00:00
1 year growth rate (%) = 49.44325780125048 calculated from 2023-10-31 00:00:00 to 2022-11-06 00:00:00
3 years growth rate (%) = 986.9440892651639 calculated from 2023-10-31 00:00:00 to 2020-11-16 00:00:00


In [181]:
# tmpAuM_6Mo = f1_DailyAuM[(range_6Mo[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_6Mo[0])]
# growthRate_6Mo = netGrowthRate(tmpAuM_6Mo.iloc[-1].AuM, tmpAuM_6Mo.iloc[0].AuM, True)

# tmpAuM_1Yr = f1_DailyAuM[(range_1Yr[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_1Yr[0])]
# growthRate_1Yr = netGrowthRate(tmpAuM_1Yr.iloc[-1].AuM, tmpAuM_1Yr.iloc[0].AuM, True)

# tmpAuM_3Yrs = f1_DailyAuM[(range_3Yrs[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_3Yrs[0])]
# growthRate_3Yrs = netGrowthRate(tmpAuM_3Yrs.iloc[-1].AuM, tmpAuM_3Yrs.iloc[0].AuM, True)

# print(f"Annualized 6 months growth rate (%) = {(growthRate_6Mo**2*100)} calculated from {tmpAuM_6Mo.iloc[-1].Date} to {tmpAuM_6Mo.iloc[0].Date}")
# print(f"Annualized 1 year growth rate (%) = {growthRate_1Yr*100} calculated from {tmpAuM_1Yr.iloc[-1].Date} to {tmpAuM_1Yr.iloc[0].Date}")
# print(f"Annualized 3 years growth rate (%) = {growthRate_3Yrs*100} calculated from {tmpAuM_3Yrs.iloc[-1].Date} to {tmpAuM_3Yrs.iloc[0].Date}")